<a href="https://colab.research.google.com/github/AbhishekDaniel1411/IBM_Data_Science_Professional_Certificate/blob/main/Capstone_project/falcon9_landing_prediction/spacex_f9_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#uncomment the following lines to install these libraries, if needed
#!pip install pandas dash
#!pip install jupyter-dash

In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
from dash import jupyter_dash

jupyter_dash.default_mode="external"

In [4]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
#import dash_html_components as html
#import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

mark_values = {0:'0', 100:'100', 500:'500', 1000:'1,000', 2000:'2,000',
               3000:'3,000', 4000:'4,000', 5000:'5,000', 6000:'6,000',
               7000:'7,000', 8000:'8,000', 9000:'9,000', 10000:'10,000'}

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options = [
                                    {'label': 'All Sites', 'value':'ALL'},
                                    {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                    {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}
                                    ],
                                value = 'ALL',
                                placeholder = 'Launch Sites',
                                searchable = True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div([dcc.Graph(id='success-pie-chart')]),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min = 0, max = 10000, step=1000,
                                                marks = mark_values,
                                                value = [max_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div([dcc.Graph(id='success-payload-scatter-chart')])
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values = 'class',
                     names = 'Launch Site',
                     title = 'Total Successful launches by sites')
        return fig
    else:
        sites_grpd = spacex_df.groupby(['Launch Site', 'class'])['Flight Number'].count().reset_index()
        temp = sites_grpd[sites_grpd['Launch Site'].str.contains(entered_site)]
        fig = px.pie(temp, values = 'Flight Number',
                     names = 'class',
                     title= f'Performance for the site {entered_site}')
        return fig



# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def get_scatter_plot(entered_site, selected_range):
    if entered_site == 'ALL':
        selected_range == [min_payload, max_payload]
        payload_mask = (spacex_df['Payload Mass (kg)'] >= selected_range[0]) & (spacex_df['Payload Mass (kg)'] <= selected_range [1])
        payload_data = spacex_df[payload_mask]
        fig = px.scatter(payload_data,
                         x = 'Payload Mass (kg)', y = 'class',
                         color = 'Booster Version Category',
                         title= 'Correlation between Payload and Success for all sites')
        return fig
    else:
        filtered_data = spacex_df[spacex_df['Launch Site'] ==  entered_site]
        payload_mask = (filtered_data['Payload Mass (kg)'] >= selected_range[0]) & (filtered_data['Payload Mass (kg)'] <= selected_range [1])
        payload_data = filtered_data[payload_mask]
        fig = px.scatter(payload_data,
                         x = 'Payload Mass (kg)', y = 'class',
                         color = 'Booster Version Category',
                         title= f'Correlation between Payload and Success for {entered_site}')
        return fig




# Run the app
if __name__ == '__main__':
    app.run_server()


Dash app running on http://127.0.0.1:8050/
